In [1]:
# Essentials
import time
import copy
from collections import OrderedDict
import random
import os
from tifffile import TiffFile
from PIL import Image, ImageOps
from pathlib import Path
from tqdm.notebook import tqdm
# Data
import numpy as np
import pandas as pd
# Plot
import matplotlib.pyplot as plt
# Torch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, Subset
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch.nn.functional as F
# Torchvision
from torchvision import datasets, transforms
import torchvision.transforms.functional as TF
# segmentation_models_pytorch
import segmentation_models_pytorch as smp
# Albumentations
import albumentations as A
from albumentations.pytorch import ToTensorV2
# Optuna
import optuna
# Local 
from unet import UNet
from LCD import LandCoverData
from dataset import *
from train import *
from utils import *
from metrics import *
from losses import *
LCD = LandCoverData()

In [2]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [3]:
train_transform = A.Compose([
    A.ToFloat(max_value=65535.0),
    A.VerticalFlip(p=0.5),
    A.HorizontalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.Rotate(limit=5, p=0.5),
    A.Blur(p=0.25),
    A.ElasticTransform(p=0.5, alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03),
    A.FromFloat(max_value=65535.0),
    A.Normalize(mean=(0.5, 0.5, 0.5, 0.5), std=(1, 1, 1, 1), max_pixel_value=65535),
    ToTensorV2()
])

test_transform = A.Compose([
    A.Normalize(mean=(0.5, 0.5, 0.5, 0.5), std=(1, 1, 1, 1), max_pixel_value=65535),
    ToTensorV2()
])

In [4]:
def def_dataset(train_dir):

    train_idx, val_idx = train_val_dataset(train_dir, val_split=0.2)
    train_set = ImageSegementationDataset(train_dir, in_channels=IN_CHANNELS, path_index=train_idx, mode='train', transforms=train_transform)
    val_set = ImageSegementationDataset(train_dir, in_channels=IN_CHANNELS, path_index=val_idx, mode='valid', transforms=test_transform)

    loader_train = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
    loader_valid = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

    data_sizes = {"train": len(loader_train), "valid": len(loader_valid)}

    return loader_train, loader_valid, data_sizes

In [5]:
def main(train_dir, segment, IN_CHANNELS, weights, n_trials, time_out):

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("Device: ", device)

    loader_train, loader_valid, data_sizes = def_dataset(train_dir)

    def objective(trial):

        if segment == 'UNET':
            model = smp.Unet(encoder_name=trial.suggest_categorical('MODEL', ['resnet18', 'resnet34', 'timm-efficientnet-b3']),in_channels=IN_CHANNELS, classes=10, activation=None)
        elif segment == 'LINKNET':
            model = smp.Linknet(encoder_name=trial.suggest_categorical('MODEL', ['resnet18', 'resnet34', 'timm-efficientnet-b3']),in_channels=IN_CHANNELS, classes=10, activation=None)
        
        optimizer_ft = optim.Adam(model.parameters(), trial.suggest_float("lr", 1e-7, 1e-2, log=True))

        scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.5)

        criterion = CombinedLoss(weights, trial.suggest_categorical('criterion', [True, False]))
        loaders = {
            "train": loader_train,
            "val": loader_valid
        }

        trainer = Trainer(model, loaders, optimizer_ft, criterion, scheduler, device)
        trainer.reset()

        trainer.run(NB_EPOCHS, reduce_on_plateau=True)
        return trainer.best_kl

    study = optuna.create_study()
    study.optimize(objective, n_trials=n_trials, timeout=time_out)

In [6]:
seed = 2021
seed_everything(seed)

train_dir='dataset/train'
SEGMENT = 'LINKNET'
NB_EPOCHS = 2
BATCH_SIZE = 12
IN_CHANNELS = 4
N_TRIALS = 2
TIME_OUT = 60 # impose nombre de secondes avant qu'il s'arrête
weights = torch.tensor([[0.0000e+00, 0.0000e+00, 2.5257e+00, 3.7227e-01, 5.4894e-01, 1.5724e+00,
        4.5365e-01, 1.6032e+01, 1.4391e+05, 5.1694e+00]])
main(train_dir, SEGMENT, IN_CHANNELS, weights, N_TRIALS, TIME_OUT)

[I 2021-03-11 18:40:03,725] A new study created in memory with name: no-name-72404b0b-c035-4360-a8a3-33b9fd9f8fbf
Device:  cpu



Epoch 1/2 Learning rate: 0.0035902081756768785
Epoch 1/2 Training Loss: 3.04816472530365 Training BCE Loss 2.500029683113098 Training Dice Loss 0.5481350421905518 Training IoU: 0.030610593158928674 Training KL: 1.835495047944056
Epoch 1/2 Validation Loss: 2.8129968643188477 Validation BCE Loss 2.2629477977752686 Validation Dice Loss 0.5500491857528687 Validation IoU: 0.03477399160579669 Validation KL: 4.166803218552554
C:\Users\inesp\AppData\Local\Programs\Python\Python38\lib\site-packages\torch\optim\lr_scheduler.py:156: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarn

[I 2021-03-11 18:40:23,880] Trial 0 finished with value: 1.5825499815953166 and parameters: {'MODEL': 'resnet18', 'lr': 0.0035902081756768785, 'criterion': False}. Best is trial 0 with value: 1.5825499815953166.

Epoch 2/2 Learning rate: 0.0035902081756768785
Epoch 2/2 Training Loss: 2.599603772163391 Training BCE Loss 2.068421423435211 Training Dice Loss 0.5311824083328247 Training IoU: 0.0800975416195942 Training KL: 1.4910419713695644
Epoch 2/2 Validation Loss: 2.6555721759796143 Validation BCE Loss 2.110816717147827 Validation Dice Loss 0.5447554588317871 Validation IoU: 0.05830240983576079 Validation KL: 1.5825499815953166



Epoch 1/2 Learning rate: 0.0002211849893707243
Epoch 1/2 Training Loss: 2.671537399291992 Training BCE Loss 2.137781262397766 Training Dice Loss 0.5337560176849365 Training IoU: 0.07578000536255333 Training KL: 0.8868688836154096
Epoch 1/2 Validation Loss: 2.79510235786438 Validation BCE Loss 2.2470345497131348 Validation Dice Loss 0.5480678081512451 Validation IoU: 0.03855032796934211 Validation KL: 9.693159445843378


[I 2021-03-11 18:40:48,970] Trial 1 finished with value: 9.693159445843378 and parameters: {'MODEL': 'resnet34', 'lr': 0.0004423699787414486, 'criterion': True}. Best is trial 0 with value: 1.5825499815953166.

Epoch 2/2 Learning rate: 0.0002211849893707243
Epoch 2/2 Training Loss: 2.5998120307922363 Training BCE Loss 2.069864511489868 Training Dice Loss 0.5299475193023682 Training IoU: 0.08257642964546434 Training KL: 0.9197489576194444
Epoch 2/2 Validation Loss: 2.7729806900024414 Validation BCE Loss 2.226407289505005 Validation Dice Loss 0.546573281288147 Validation IoU: 0.0308849096574715 Validation KL: 10.709428453236939
